# Bronze Ingestion – Raw E-commerce Events

This notebook ingests raw e-commerce event data into the Bronze layer.
No transformations are applied at this stage.


In [0]:
SOURCE_TABLE = "workspace.default.events"
BRONZE_TABLE = "workspace.default.bronze_events"


In [0]:
df_events = spark.table(SOURCE_TABLE)
display(df_events.limit(20))


In [0]:
(
    df_events
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(BRONZE_TABLE)
)


In [0]:
spark.table(BRONZE_TABLE).limit(20)


In [0]:
print("Source rows :", df_events.count())
print("Bronze rows :", spark.table(BRONZE_TABLE).count())


# Bronze ingestion completed successfully
# Source  : workspace.default.events
# Target  : workspace.default.bronze_events
# Strategy: Full overwrite, no transformations


In [0]:
SOURCE_TABLE = "workspace.default.bronze_events"
